# Masakhane - Traduction Automatique pour les Langues Africaines (avec JoeyNMT)

## Note avant de commencer :
### - L'idée est que vous puissiez y apporter des modifications minimales afin d'obtenir QUELQUES résultats pour votre propre corpus de traduction. 

### - Le tl;dr : Allez aux commentaires **"TODO "** qui vous indiqueront ce qu'il faut mettre à jour pour que tout fonctionne bien.

### - Si vous voulez avoir une idée de ce que vous êtes en train de faire, veuillez lire le texte et consulter les liens.

### - Avec 100 epochs, l'exécution dans Google Colab devrait prendre environ 7 heures.

### - Une fois que vous avez obtenu un bon résultat pour votre langue, veuillez joindre et envoyer par courriel votre notebook qui en est à l'origine à masakhanetranslation@gmail.com.

### - Si vous en avez l'occasion, il serait fantastique de faire un bref historique de votre langue. Voir des exemples dans  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Récupérer vos données et créer un corpus parallèle

Si vous souhaitez utiliser les données JW300 référencées sur le site de Masakhane ou dans notre repo GitHub, vous pouvez utiliser `opus-tools` pour convertir les données dans un format approprié. `opus_read` de ce package fournit un outil pratique pour lire les fichiers XML natifs alignés et les convertir au format TMX. L'outil peut également être utilisé pour récupérer à la volée des fichiers pertinents d'OPUS ainsi que pour filtrer les données si nécessaire. [Lire la documentation](https://pypi.org/project/opustools-pkg/) pour plus de détails.

Une fois que vous avez vos fichiers de corpus au format TMX (une structure XML qui comprend les phrases dans votre langue cible et votre langue source dans un seul fichier), nous recommandons de les lire dans un dataframe Pandas. Heureusement, Jade a écrit un paquet ingénieux `tmx2dataframe` qui convertit votre fichier tmx en un dataframe Pandas. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# TODO: Définissez vos langues source et cible. 
# N'oubliez pas que ces langues utilisent traditionnellement les codes de langue que vous trouverez ici :
# Ces suffixes deviendront également ceux de tous les fichiers de vocabulaire et de corpus utilisés dans l'ensemble du projet.
import os
source_language = "en"
target_language = "xh" 
lc   = False      # Si True, mettre les données en minuscules.
seed = 42         # Seed aléatoire pour le mélange des données.
tag  = "baseline" # Donner un nom unique à votre dossier - ceci afin de vous assurer que vous ne réécrivez pas les modèles que vous avez déjà soumis

os.environ["src"] = source_language # Les définir en bash également, puisque nous utilisons souvent des scripts bash
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# Ceci va le sauvegarder à la place dans un dossier de notre Gdrive !
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [ ]:
!echo $gdrive_path

In [ ]:
# Installer les outils opus
! pip install opustools-pkg

In [ ]:
# Téléchargement de notre corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# Extraction du fichier du corpus
! gunzip JW300_latest_xml_$src-$tgt.xml.gz

In [ ]:
# Téléchargement de l'ensemble de test global.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

In [ ]:
# Lecture des données de test à filtrer à partir des splits (lots) train et dev.
# Conservation de la portion en anglais dans le lot pour un contrôle rapide du filtrage.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

In [ ]:
import pandas as pd

# Conversion du fichier TMX en dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collecte des numéros de ligne de la partie source pour sauter les mêmes lignes dans la partie cible.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Omission des phrases qui sont contenues dans le jeu de test.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Ajout au corpus uniquement si la source correspondante n'a pas été omise.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Chargement des données et suppression de {}/{} lignes car contenues dans l\'ensemble de test.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# Si vous obtenez "TypeError: data argument can't be an iterator", c'est dû à votre version zip, Exécutez ceci:
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

## Pré-traitement et exportation

C'est généralement une bonne idée de supprimer les traductions en double et les traductions contradictoires du corpus. En pratique, ces corpus publics en comportent un certain nombre qui doivent être nettoyés.

De plus, nous allons diviser nos données en dev/test/train et les exporter vers le système de fichiers.

In [ ]:
# Suppression des traductions en double
df_pp = df.drop_duplicates()

# Suppression des traductions contradictoires
# (ceci est facultatif et vous pouvez le commenter en 
# fonction de la taille de votre corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Mélange des données pour éliminer les biais dans la sélection des ensembles de données.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

In [ ]:
# Installaton de fuzzy wuzzy pour supprimer les phrases
# "presque en double" dans les jeux de test et d'entraînement.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np
from os import cpu_count
from functools import partial
from multiprocessing import Pool


# Réinitialisation de l'index de l'ensemble d'apprentissage après le filtrage précédent
df_pp.reset_index(drop=False, inplace=True)

# Suppression des échantillons de l'ensemble de données d'apprentissage
# s'ils "se chevauchent presque" avec les échantillons de l'ensemble de test.

# Fonction de filtrage. Ajustement du tampon (pad) pour réduire les correspondances candidates
# à une certaine longueur de caractères de l'échantillon donné.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

In [ ]:
start_time = time.time()
### itérer sur les lignes du dataframe pandas n'est pas recommandé, 
### utilisons le traitement multiple pour appliquer la fonction.

with Pool(cpu_count()-1) as pool:
    scores = pool.map(partial(fuzzfilter, candidates=list(en_test_sents), pad=5), df_pp['source_sentence'])
hours, rem = divmod(time.time() - start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("done in {}h:{}min:{}seconds".format(hours, minutes, seconds))

# Filtrage des "échantillons qui se recouvrent quasiment".
df_pp = df_pp.assign(scores=scores)
df_pp = df_pp[df_pp['scores'] < 95]

In [ ]:
# Cette section effectue la répartition entre train/dev pour les corpus parallèles
# et les enregistre dans des fichiers séparés.
# Nous utilisons 1000 pour dev test et le jeu de test donné.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Facultatif : mettre les corpus en minuscules - cela facilitera la généralisation, mais sans la casse appropriée.
if lc:  # Julia: rendre la mise en minuscule facultative
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: les jeux de test ont déjà été générés
dev = df_pp.tail(num_dev_patterns) # Herman: Erreur dans l'original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Ajout partout de `header=False`
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Traitement problématique des guillemets.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Vérification du format ci-dessous. Il ne doit pas y avoir de guillemets supplémentaires ou de caractères bizarres.
! head train.*
! head dev.*



---


## Installation de JoeyNMT

JoeyNMT est un paquet NMT (ou TAN - Traduction Automatique Neurale) simple et minimaliste, utile pour l'apprentissage et l'enseignement. Vous pouvez consulter la documentation de JoeyNMT [ici](https://joeynmt.readthedocs.io)  

In [ ]:
# Installation de JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .
# Installation de Pytorch avec le support GPU v1.7.1.
! pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# Prétraitement des données en jetons (tokens) de sous-mots BPE

- L'une des améliorations les plus performantes pour les langues agglutinantes (une caractéristique de la plupart des langues bantoues) est l'utilisation de la tokenisation BPE. [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- Il a également été démontré qu'en optimisant le nombre de codes BPE, nous améliorons considérablement les résultats pour les langues à faibles ressources. [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Ci-dessous, nous avons les scripts pour faire la tokenisation BPE de nos données. Nous utilisons 4000 tokens comme recommandé par [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). Vous n'avez besoin de rien changer. Il suffit juste d'exécuter le programme ci-dessous. 

In [ ]:
# Une des améliorations considérables des performances du NMT a été 
# l'utilisation d'une méthode différente de tokenisation. 
# Habituellement, NMT effectue une tokénisation par mots. Cependant, l'utilisation d'une méthode 
# appelée BPE (Byte Pair Encoding) a permis d'améliorer considérablement les performances.

# Effectuer un NMT de sous-mots
from os import path
os.environ["src"] = source_language # Les définir en bash également, puisque nous utilisons souvent des scripts bash
os.environ["tgt"] = target_language

# Apprentissage des BPE sur les données de formation.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Application des splits (fractions) BPE aux données de développement et de test.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Création d'un répertoire et déplacement de toutes les données qui nous intéressent vers le bon emplacement.
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Déplacement également de tout ce qui nous intéresse vers un emplacement 
# monté dans google drive à gdrive_path (pertinent si vous travaillez en colab).
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Création du vocabulaire à l'aide de build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Quelques résultats
! echo "BPE Xhosa Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

In [ ]:
# Déplacement également de tout ce qui nous intéresse vers un emplacement 
# monté dans google drive  à gdrive_path (pertinent si vous travaillez en colab).
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Création de la configuration de JoeyNMT

JoeyNMT nécessite une configuration yaml. Nous fournissons un modèle ci-dessous. 
Nous avons également défini un certain nombre de valeurs par défaut, avec lesquelles vous pouvez jouer !

- Nous avons utilisé l'architecture Transformer 
- Nous avons fixé notre taux de dropout (décrochage) à un niveau raisonnablement élevé : 0,3 (recommandé dans le document  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Des choses qui valent la peine de jouer avec :
- La taille du lot ou batch (qu'il est également recommandé de modifier pour les langues à faibles ressources)
- Le nombre d'époques (nous l'avons fixé à 30 pour qu'il s'exécute en une heure environ, à des fins de test).
- Les caractéristiques du décodeur (beam_size, alpha)
- Les métriques d'évaluation (BLEU versus Crhf4)

In [ ]:
# Ceci crée le fichier de configuration de notre système JoeyNMT. Étant donné que ce fichier peut sembler 
# trop volumineux, nous vous fournissons quelques paramètres utiles à mettre à jour.
# (Vous pouvez bien sûr jouer avec tous les paramètres si vous le voulez !)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Création de la configuration
# Note: TODO -> À faire
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # si décommenté, charger un modèle pré-entraîné à partir de ce point de contrôle (checkpoint)
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: Essaie de basculer du plateau au scheduler (planificateur) Noam
    patience: 5                     # Pour le plateau : diminuer le taux d'apprentissage par le decrease_factor (facteur de diminution) si le score de validation ne s'est pas amélioré pendant ce nombre de tours de validation..
    learning_rate_factor: 0.5       # Facteur pour l'ordonnanceur Noam (utilisé avec le Transformer)
    learning_rate_warmup: 1000      # Étapes de démarrage (warmup steps) pour le scheduler Noam (utilisé avec le Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Diminuer pour jouer et vérifier le fonctionnement. Une trentaine de minutes suffisent pour vérifier si le système fonctionne.
    validation_freq: 1000          # TODO: Défini à au moins une fois par époque (epoch).
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # TODO: À mettre à True si vous voulez écraser les modèles éventuellement existants. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Augmenter à 8 pour des données plus importantes.
        embeddings:
            embedding_dim: 256   # TODO: Augmenter à 512 pour des données plus importantes.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Augmenter à 512 pour des données plus importantes.
        ff_size: 1024            # TODO: Augmenter à 2048 pour des données plus importantes.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Augmenter à 8 pour des données plus importantes.
        embeddings:
            embedding_dim: 256    # TODO: Augmenter à 512 pour des données plus importantes.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Augmenter à 512 pour des données plus importantes.
        ff_size: 1024            # TODO: Augmenter à 2048 pour des données plus importantes.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Formation du modèle

Cette seule ligne de joeynmt exécute l'entraînement en utilisant la configuration que nous avons faite ci-dessus.

In [ ]:
# Formation du modèle
# Vous pouvez appuyer sur Ctrl-C pour interrompre. Ensuite, exécutez 
# la cellule suivante pour sauvegarder vos points de contrôle (checkpoints) ! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

In [ ]:
# Copie des modèles créés depuis le stockage de l'ordinateur portable vers Google Drive pour un stockage persistant. 
!cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

In [ ]:
# Sortie de notre précision de validation (validation accuracy)
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

In [ ]:
# Test du modèle
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"